# CAL FIRE DATASET ANALYSIS
Break Through Tech AI, Team Snowflake 1A 

Load datasets from SQL into Pandas dataframes.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
SELECT * FROM FIRE.PUBLIC.CALFIRE_INCIDENTS

In [ ]:
SELECT * FROM FIRE.PUBLIC.CALFIRE

In [ ]:
SELECT * FROM FIRE.PUBLIC.CALFIRE_DAMAGE

In [ ]:
fire_incidents_df = load_incident.to_pandas()

### Processing Fire Incidents DF

In [ ]:
fire_incidents_rename = {
    'NAME': 'INCIDENT_NAME' 
}
fire_incidents_df.rename(columns=fire_incidents_rename, inplace=True)

### Processing Fire DF 

In [ ]:
fire_df = load_df.to_pandas()

In [ ]:
total_count = len(fire_df)
total_count

In [ ]:
missing_counts = fire_df.isnull().sum() 
missing_counts

In [ ]:
fire_df['TIME_TO_EXTINGUISH'] = (fire_df['INCIDENT_DATE_EXTINGUISHED'] - fire_df['INCIDENT_DATE_CREATED']).dt.total_seconds()/3600
fire_df[['INCIDENT_NAME', 'TIME_TO_EXTINGUISH']]

In [ ]:
fire_df['YEAR'] = fire_df['INCIDENT_DATE_CREATED'].dt.year

In [ ]:
fire_df

### Fire Damage Dataset Processing

In [ ]:
fire_damage_df_snowflake = load_damage.to_df()
fire_damage_df_snowflake

In [ ]:
fire_damage_df = fire_damage_df_snowflake.to_pandas()
fire_damage_df

In [ ]:
rename_dict = {
    'OBJECTID': 'ID',
    '* Damage': 'DAMAGE',
    '* Street Number': 'STREET_NUM',
    '* Street Name': 'STREET_NAME',
    '* Street Type (e.g. road, drive, lane, etc.)': 'STREET_TYPE',
    'Street Suffix (e.g. apt. 23, blding C)': 'STREET_SUFFIX',
    '* City': 'CITY',
    'STATE': 'STATE',
    'ZIP_CODE': 'ZIP_CODE',
    '* CAL FIRE Unit': 'CAL_FIRE_UNIT',
    'COUNTY': 'COUNTY',
    'COMMUNITY': 'COMMUNITY',
    'BATTALION': 'BATTALION',
    '* Incident Name': 'INCIDENT_NAME',
    'Incident Number (e.g. CAAEU 123456)': 'INCIDENT_NUM',
    'INCIDENT_START_DATE': 'START_DATE',
    'HAZARD_TYPE': 'HAZARD_TYPE',
    'If Affected 1-9% - Where did fire start?': 'FIRE_ORIGIN',
    'If Affected 1-9% - What started fire?': 'FIRE_CAUSE',
    'STRUCTURE_DEFENSE_ACTIONS_TAKEN': 'DEFENSE_ACTIONS',
    '* Structure Type': 'STRUCTURE_TYPE',
    'STRUCTURE_CATEGORY': 'STRUCTURE_CATEGORY',
    '# Units in Structure (if multi unit)': 'UNITS_STRUCTURE',
    '# of Damaged Outbuildings < 120 SQFT': 'DAMAGED_OUTBUILDINGS',
    '# of Non Damaged Outbuildings < 120 SQFT': 'NON_DAMAGED_OUTBUILDINGS',
    '* Roof Construction': 'ROOF_TYPE',
    '* Eaves': 'EAVES_TYPE',
    '* Vent Screen': 'VENT_SCREEN',
    '* Exterior Siding': 'SIDING',
    '* Window Pane': 'WINDOW_PANE',
    '* Deck/Porch On Grade': 'DECK_ON_GRADE',
    '* Deck/Porch Elevated': 'DECK_ELEVATED',
    '* Patio Cover/Carport Attached to Structure': 'PATIO_COVER',
    '* Fence Attached to Structure': 'FENCE_ATTACHED',
    'Distance - Propane Tank to Structure': 'PROPANE_DISTANCE',
    'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT': 'RESIDENCE_DISTANCE',
    'Fire Name (Secondary)': 'FIRE_NAME_SECONDARY',
    'APN (parcel)': 'APN',
    'Assessed Improved Value (parcel)': 'IMPROVED_VALUE',
    'Year Built (parcel)': 'YEAR_BUILT',
    'Site Address (parcel)': 'SITE_ADDRESS',
    'GLOBALID': 'GLOBAL_ID',
    'LATITUDE': 'LATITUDE',
    'LONGITUDE': 'LONGITUDE',
    'X': 'X_COORD',
    'Y': 'Y_COORD'
}

fire_damage_df.rename(columns=rename_dict, inplace=True)

In [ ]:
print(fire_damage_df['DAMAGE'].unique())

In [ ]:
print(len(fire_damage_df['INCIDENT_NAME'].unique()))

In [ ]:
damage_counts = (
    fire_damage_df.groupby(['INCIDENT_NAME', 'DAMAGE'])
    .size()  # or use .count() if you want to count non-null values in another column
    .reset_index(name='COUNT')  # Resetting index to convert the result into a DataFrame
)

print(damage_counts)

In [ ]:
import pandas as pd
damage_pivot = damage_counts.pivot_table(
    index='INCIDENT_NAME', 
    columns='DAMAGE', 
    values='COUNT', 
    fill_value=0  # Fill NaN with 0
).reset_index()
damage_df = pd.DataFrame(damage_pivot)

In [ ]:
damage_df['INCIDENT_NAME'] += ' Fire'

In [ ]:
damage_df

In [ ]:
none_values_per_column = damage_df.isna().sum()
print(none_values_per_column)

## Combining the Datasets

In [ ]:
import matplotlib.pyplot as plt
# which counties get the most wildfires
wildfire_counties = fire_df['INCIDENT_COUNTY'].value_counts().sort_index()
wildfire_counties_2 = fire_incidents_df['COUNTIES'].value_counts().sort_index()

# print(wildfire_counties)
# print(wildfire_counties.nlargest(30))
# print(wildfire_counties_2.nlargest(30))


top_wildfire_counties = wildfire_counties.nlargest(30)
top_wildfire_counties_2 = wildfire_counties_2.nlargest(30)

top_wildfire_counties.plot(kind='bar', color='skyblue', label='calfire')
top_wildfire_counties_2.plot(kind='bar', label='incidents')
plt.title('Number of Wildfires per County')
plt.xlabel('County')
plt.ylabel('Number of Wildfires')
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.legend()
plt.show()

In [ ]:
SELECT * FROM FIRE.PUBLIC.CAL_LAT_LON

In [ ]:
cal_lat_lon_df = loading_lat_lon.to_pandas()
cal_lat_lon_df

In [ ]:
fire_df_final = fire_df[['INCIDENT_NAME', 'INCIDENT_DATE_CREATED', 'INCIDENT_ADMINISTRATIVE_UNIT', 'INCIDENT_COUNTY', 'INCIDENT_ACRES_BURNED', 'INCIDENT_CONTAINMENT', 'TIME_TO_EXTINGUISH']]
fire_df_final

In [ ]:
# Rename columns if necessary
cal_lat_lon_df.rename(columns={"county": "COUNTY"}, inplace=True)

# Perform the merge
fire_df_final = fire_df_final.merge(
    cal_lat_lon_df[['COUNTY', 'LATITUDE', 'LONGITUDE']],
    on='INCIDENT_COUNTY',  # Common column
    how='left'  # 'left' keeps all rows from fire_df_final
)

# Inspect the merged DataFrame
print(fire_df_final.head())



In [ ]:
fire_df_final = cal_lat_lon_df[['LATITUDE', 'LONGITUDE']]
fire_df_final

In [ ]:
damage_df

In [ ]:
missing_counts_2 = fire_incidents_df_final.isnull().sum() 
missing_counts_2

In [ ]:
fire_incidents_final_df = fire_incidents_df[['INCIDENT_NAME', 'COUNTIES', 'CREWSINVOLVED', 'DOZERS', 'ENGINES', 'FATALITIES', 'HELICOPTERS', 'INJURIES', 'PERSONNELINVOLVED', 'STRUCTURESDAMAGED', 'STRUCTURESDESTROYED', 'STRUCTURESEVACUATED', 'STRUCTURESTHREATENED']]
fire_incidents_final_df

In [ ]:
fire_df_final['INCIDENT_COUNTY'] = fire_df_final['INCIDENT_COUNTY'].str.split(',').str[0].str.strip()

In [ ]:
# scatter plots (for a couple of counties, acres burned vs time to extinguish)
# box plots (distribution of like acres burned, time to extinguish)

In [ ]:
# fire_df_final aggregate over county 
fire_df_agg = fire_df_final.groupby('INCIDENT_COUNTY').agg(
    MIN_ACRES_BURNED=('INCIDENT_ACRES_BURNED', 'min'),
    MAX_ACRES_BURNED=('INCIDENT_ACRES_BURNED', 'max'),
    AVG_ACRES_BURNED=('INCIDENT_ACRES_BURNED', 'mean'),
    STDEV_ACRES_BURNED=('INCIDENT_ACRES_BURNED', 'std'),
    # MIN_TIME_TO_EXTINGUISH=('TIME_TO_EXTINGUISH', 'min'),
    MAX_TIME_TO_EXTINGUISH=('TIME_TO_EXTINGUISH', 'max'),
    AVG_TIME_TO_EXTINGUISH=('TIME_TO_EXTINGUISH', 'mean'),
    STDEV_TIME_TO_EXTINGUISH=('TIME_TO_EXTINGUISH', 'std')
).reset_index()
fire_df_agg

In [ ]:
missing_counts_2 = fire_incidents_final_df.isnull().sum() 
missing_counts_2

In [ ]:
filtered_df = fire_incidents_final_df.dropna(subset=['CREWSINVOLVED'])
filtered_df

In [ ]:
# re_incidents_df[['INCIDENT_NAME', 'COUNTIES', 'CREWSINVOLVED', 'DOZERS', 'ENGINES', 'FATALITIES', 'HELICOPTERS', 'INJURIES', 'PERSONNELINVOLVED', 'STRUCTURESDAMAGED', 'STRUCTURESDESTROYED', 'STRUCTURESEVACUATED', 'STRUCTURESTHREATENED']
# CREWSINVOLVED, DOZERS, ENGINES, HELICOPTERS, PERSONNELINVOLVED
# fire_incidents_final_df
# fire_df_final aggregate over county 
fire_incidents_agg = fire_incidents_final_df.groupby('COUNTIES').agg(
    AVG_CREWSINVOLVED=('CREWSINVOLVED', 'mean'),
    AVG_DOZERS=('DOZERS', 'mean'),
    AVG_ENGINERS=('ENGINES', 'mean'),
    AVG_HELICOPERS=('HELICOPTERS', 'mean'),
    AVG_PERSONNELINVOLVED=('PERSONNELINVOLVED', 'mean'),
).reset_index()
# fire_incidents_agg
fire_incidents_agg_filled = fire_incidents_agg.fillna(0)
fire_incidents_agg_filled.rename(columns={'COUNTIES': 'INCIDENT_COUNTY'}, inplace=True)
fire_incidents_agg_filled

In [ ]:
session = get_active_session()
session.write_pandas(
    combined_agg_df,
    table_name='cal_fire_combined_agg_cleaned',
    auto_create_table=True
   )

In [ ]:
combined_agg_df = fire_df_agg.merge(fire_incidents_agg_filled, on='INCIDENT_COUNTY', how='left')
combined_agg_df

In [ ]:
session = get_active_session()
session.write_pandas(
    combined_agg_df,
    table_name='cal_fire_combined_agg_cleaned',
    auto_create_table=True
   )

In [ ]:
combined_agg_df.to_csv('calfire_agg_v0.csv', index=False)

In [ ]:
# combined_df = fire_df_final.merge(damage_df, on='INCIDENT_NAME', how='left')
# combined_df = combined_df.merge(fire_incidents_final_df, on='INCIDENT_NAME', how='left')
# combined_df

In [ ]:
combined_df = fire_df_final.merge(fire_incidents_final_df, on='INCIDENT_NAME', how='left')
combined_df

In [ ]:
print(combined_df.columns)

In [ ]:
fire_unique = fire_df_final['INCIDENT_NAME'].unique()
fire_incidents_unique = fire_incidents_final_df['INCIDENT_NAME'].unique() # scuffed
damage_unique = damage_df['INCIDENT_NAME'].unique()

In [ ]:
none_values_per_column_incidents = fire_incidents_final_df.isna().sum()
print(none_values_per_column_incidents)
print(len(fire_incidents_final_df))

In [ ]:
overlap_12 = set(fire_unique).intersection(fire_incidents_unique)
overlap_13 = set(fire_unique).intersection(damage_unique)
overlap_23 = set(fire_incidents_unique).intersection(damage_unique)
overlap_all = set(fire_unique).intersection(fire_incidents_unique, damage_unique)

print("Overlap between df1 and df2:", len(overlap_12), overlap_12)
print("Overlap between df1 and df3:", len(overlap_13), overlap_13)
print("Overlap between df2 and df3:", len(overlap_23), overlap_23)
print("Overlap between df1, df2, and df3:", len(overlap_all), overlap_all)

## EDA 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.figure(figsize=(10, 6))
wildfire_counts = fire_df['YEAR'].value_counts().sort_index()
# print(wildfire_counts)
wildfire_counts.plot(kind='bar', color='skyblue')
plt.title('Number of Wildfires per Year')
plt.xlabel('Year')
plt.ylabel('Number of Wildfires')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
# Ensure there are no NaN values in the columns you're plotting
scatter_df = fire_df.dropna(subset=['INCIDENT_ACRES_BURNED', 'TIME_TO_EXTINGUISH'])

plt.figure(figsize=(10, 6))
plt.scatter(scatter_df['INCIDENT_ACRES_BURNED'], scatter_df['TIME_TO_EXTINGUISH'], alpha=0.5)
plt.title('Acres Burned vs. Time to Extinguish')
plt.xlabel('Acres Burned')
plt.ylabel('Time to Extinguish (hours)')
plt.grid(True)
plt.show()


In [ ]:
# Select numerical columns
numeric_cols = fire_df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = fire_df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=90)
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()


In [ ]:
# Assuming 'FIRE_CAUSE' is a column in your fire_damage_df DataFrame
cause_counts = fire_damage_df['FIRE_CAUSE'].value_counts().nlargest(10)

plt.figure(figsize=(10, 6))
cause_counts.plot(kind='bar', color='orange', edgecolor='black')
plt.title('Top Causes of Fires')
plt.xlabel('Cause')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
annual_acres = fire_df.groupby('YEAR')['INCIDENT_ACRES_BURNED'].sum().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(annual_acres['YEAR'], annual_acres['INCIDENT_ACRES_BURNED'], marker='o')
plt.title('Total Acres Burned Over Years')
plt.xlabel('Year')
plt.ylabel('Total Acres Burned')
plt.grid(True)
plt.show()


In [ ]:
fire_df['MONTH'] = fire_df['INCIDENT_DATE_CREATED'].dt.month_name()
monthly_counts = fire_df['MONTH'].value_counts().reindex([
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
])

plt.figure(figsize=(12, 6))
monthly_counts.plot(kind='bar', color='green', edgecolor='black')
plt.title('Number of Fires by Month')
plt.xlabel('Month')
plt.ylabel('Number of Fires')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
# Assuming 'LATITUDE' and 'LONGITUDE' are columns in fire_damage_df

plt.figure(figsize=(10, 8))
plt.scatter(fire_damage_df['LONGITUDE'], fire_damage_df['LATITUDE'], alpha=0.5, s=10, c='red')
plt.title('Geographic Distribution of Fires')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()


In [ ]:
# which counties get the most wildfires
wildfire_counties = fire_df['INCIDENT_COUNTY'].value_counts().sort_index()
# print(wildfire_counties)
print(wildfire_counties.nlargest(30))

top_wildfire_counties = wildfire_counties.nlargest(30)

top_wildfire_counties.plot(kind='bar', color='skyblue')
plt.title('Number of Wildfires per County')
plt.xlabel('County')
plt.ylabel('Number of Wildfires')
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
fire_df

In [ ]:
import pandas as pd
# Identify non-numeric columns
non_numeric_columns = fire_df.select_dtypes(exclude=['number']).columns

# Apply one-hot encoding to non-numeric columns
fire_df_encoded = pd.get_dummies(fire_df, columns=non_numeric_columns, drop_first=True)

for col in fire_df_encoded.select_dtypes(include=['number']).columns:
    fire_df_encoded[col].fillna(fire_df_encoded[col].mean(), inplace=True)

# For non-numeric (encoded) columns, fill missing values with the mode
for col in fire_df_encoded.select_dtypes(exclude=['number']).columns:
    fire_df_encoded[col].fillna(fire_df_encoded[col].mode()[0], inplace=True)



# Step 1: Check for non-numeric columns
non_numeric_columns = fire_df_encoded.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:", list(non_numeric_columns))

# Step 2: Check for columns with missing values
missing_values = fire_df_encoded.isnull().sum()
columns_with_missing_values = missing_values[missing_values > 0].index
print("Columns with missing values:", list(columns_with_missing_values))


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)

# Step 2: Fit the KMeans model to the data (without the labels)
kmeans.fit(fire_df_encoded)

# Step 3: Use the predict() method to assign cluster labels to each data point
cluster_labels = kmeans.predict(fire_df_encoded)

# Step 4: Add the cluster labels to the original Pandas DataFrame
fire_df_encoded['cluster'] = cluster_labels

# Step 5: Inspect the DataFrame with the new cluster labels
print(fire_df_encoded.head())

In [ ]:
# fire_df.drop
kmeans = KMeans(n_clusters=3, random_state=42)

# Step 2: Fit the KMeans model to the data (without the labels)
kmeans.fit(fire_df_encoded)

# Step 3: Use the predict() method to assign cluster labels to each data point
cluster_labels = kmeans.predict(fire_df_encoded)

# Step 4: Add the cluster labels to the original Pandas DataFrame
fire_df_encoded['cluster'] = cluster_labels

# Step 5: Inspect the DataFrame with the new cluster labels
print(fire_df_encoded.head())

In [ ]:
# # fire_df_encoded['Cluster'] = kmeans.labels_

# # # Step 2: Group the data by cluster and calculate the mean for each feature
# # cluster_means = fire_df_encoded.groupby('Cluster').mean()

# # # Step 3: Print the mean values of each feature by cluster
# # print("Average values for each cluster:")
# # print(cluster_means)


# import pandas as pd
# from IPython.display import display

# # Assuming 'kmeans' is the fitted KMeans model and 'fire_df_encoded' is the preprocessed DataFrame

# # Step 1: Assign the cluster labels to the DataFrame
# fire_df_encoded['Cluster'] = kmeans.labels_

# # Step 2: Group the data by cluster and calculate the mean for each feature
# cluster_means = fire_df_encoded.groupby('Cluster').mean()

# # Step 3: Reset index to make the cluster labels a column
# cluster_means.reset_index(inplace=True)

# # Step 4: Display the means as a DataTable
# print("Average values for each cluster:")
# display(cluster_means)
# display(cluster_means.iloc[:, :7])  # Including the 'Cluster' column plus the first 6 feature columns

import pandas as pd
from IPython.display import display

# Assuming 'kmeans' is the fitted KMeans model and 'fire_df_encoded' is the preprocessed DataFrame

# Step 1: Assign the cluster labels to the DataFrame
fire_df_encoded['Cluster'] = kmeans.labels_

# Step 2: Group the data by cluster and calculate the mean for each feature
cluster_means = fire_df_encoded.groupby('Cluster').mean()

# Step 3: Reset index to make the cluster labels a column
cluster_means.reset_index(inplace=True)

# Step 4: Display the means as a DataFrame
print("Average values for each cluster:")

# Creating a new DataFrame to hold the cluster averages
cluster_averages_df = pd.DataFrame(cluster_means)

# Displaying the DataFrame
cluster_averages_df

# Optionally, display only the first 6 columns including 'Cluster'
# display(cluster_averages_df.iloc[:, :7])  # Showing first 6 columns plus 'Cluster' column


## only include columns specific to impact
## do cleaning
## add clustering